In [17]:
import re

def parse_log_file(file_path):
    """讀取日誌文件並返回內容"""
    with open(file_path, 'r') as file:
        log_content = file.read()
    return log_content

def extract_messages(log_content):
    """使用正則表達式提取每個消息塊"""
    pattern = r'/\* .*? \*/\n(.*?)\n\.'
    messages = re.findall(pattern, log_content, re.DOTALL)
    return messages

def parse_message(message):
    """解析單個消息，提取發送者、消息類型和關鍵數據"""
    # 提取消息方向（Host 或 Equipment）
    header_pattern = r'/\* .*? Dir=(\d) .*? \*/'
    header_match = re.search(header_pattern, message)
    if header_match:
        direction = 'Host' if header_match.group(1) == '1' else 'Equipment'
    else:
        direction = 'Unknown'

    # 提取消息名稱和代碼
    name_pattern = r'(\w+): \'(\w+)\''
    name_match = re.search(name_pattern, message)
    if name_match:
        message_type = name_match.group(1) + ' (' + name_match.group(2) + ')'
    else:
        message_type = 'Unknown'

    # 提取關鍵數據（僅提取 <A> 標籤中的數據）
    data_pattern = r'<A \[\d+\] \'(.*?)\'>'
    data_matches = re.findall(data_pattern, message)
    key_data = ', '.join(data_matches) if data_matches else 'No data'

    return direction, message_type, key_data

def generate_table(messages):
    """將解析的數據組織成表格格式"""
    table = []
    for i, message in enumerate(messages, 1):
        direction, message_type, key_data = parse_message(message)
        table.append([str(i), direction, message_type, key_data])
    return table

def print_table(table):
    """以直立表格形式輸出結果"""
    print('| 步驟 | 發送者    | 消息類型           | 關鍵數據                                                                 |')
    print('|------|-----------|--------------------|--------------------------------------------------------------------------|')
    for row in table:
        print(f'| {row[0]:<4} | {row[1]:<9} | {row[2]:<18} | {row[3]:<72} |')

file_path = 'pp.log'
def main(file_path):
    """主函數，執行日誌解析和表格生成"""
    log_content = parse_log_file(file_path)
    messages = extract_messages(log_content)
    table = generate_table(messages)
    print_table(table)

# 調用主函數，傳入日誌文件路徑
# 例如：main('path/to/your/logfile.txt')